In [ ]:
# take two on features
import rasterio
import numpy as np
from scipy.ndimage import generic_filter

dem = r"D:\ASOML\Sierras\features\ASOML_dem_slope_60_albn83.tif"
# Load your DEM
with rasterio.open(dem) as src:
    dem = src.read(1, masked=True)  # Read as masked array to handle nodata
    profile = src.profile
    print("dem read in")

# Define a function that handles masked arrays for std
def std_func(values):
    values = values[~np.isnan(values)]
    return np.std(values) if len(values) > 0 else np.nan

print("function defined")
# Apply 3x3 window standard deviation
std_dem = generic_filter(dem.filled(np.nan), std_func, size=3)
print("window created")

# Mask the result where original DEM was masked
std_dem = np.ma.array(std_dem, mask=dem.mask)
print("file masked")

# Update profile for output
profile.update(dtype=rasterio.float32, nodata=np.nan)
print("profile")

# Save output
with rasterio.open("slope_std_3x3.tif", "w", **profile) as dst:
    dst.write(std_dem.astype(rasterio.float32), 1)


In [1]:
import rasterio
from rasterio.transform import xy
import numpy as np

# Path to your reference raster
out_raster_path =  r"D:\ASOML\Rockies\features\longitude_raster.tif"
dem = r"D:/ASOML/Rockies/features/ASO_CON_dem_albn83_60m.tif"
# Open reference raster
with rasterio.open(dem) as src:
    height, width = src.height, src.width
    transform = src.transform
    crs = src.crs
    print("raster read in")

    # Get X (longitude) coordinates for each column index
    col_indices = np.arange(width)
    row_index = 0  # any row works since X doesn't change with Y
    xs, _ = xy(transform, [row_index] * width, col_indices, offset='center')
    print("cols")

    # Broadcast to all rows
    longitudes = np.tile(xs, (height, 1)).astype(np.float32)
    print("broadcasting")

    # Write to raster
    profile = src.profile
    profile.update(dtype='float32', count=1, compress='lzw')

    with rasterio.open(out_raster_path, 'w', **profile) as dst:
        dst.write(longitudes, 1)

raster read in
cols
broadcasting


In [2]:
import rasterio
import numpy as np
from rasterio.transform import xy

out_raster_path = r"D:\ASOML\Rockies\features\latitude_raster.tif"
dem = r"D:/ASOML/Rockies/features/ASO_CON_dem_albn83_60m.tif"
with rasterio.open(dem) as src:
    height, width = src.height, src.width
    transform = src.transform
    crs = src.crs

    # Get Y (latitude) coordinates for each row index
    row_indices = np.arange(height)
    col_index = 0  # doesn't matter, since Y doesn't change with X
    _, ys = xy(transform, row_indices, [col_index] * height, offset='center')

    # Broadcast to all columns to make a 2D array
    latitudes = np.tile(np.array(ys).reshape(-1, 1), (1, width)).astype(np.float32)

    # Write to raster
    profile = src.profile
    profile.update(dtype='float32', count=1, compress='lzw')

    with rasterio.open(out_raster_path, 'w', **profile) as dst:
        dst.write(latitudes, 1)
